<img src="https://github.com/NTMC-Community/MatchZoo/blob/2.0/artworks/matchzoo-logo.png?raw=True" alt="logo" style="width:600px;float: center"/>

In [1]:
import matchzoo as mz

/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


# Prepare Data

In [2]:
train_data_pack = mz.datasets.wiki_qa.load_data(stage='train', task='ranking')
test_data_pack = mz.datasets.wiki_qa.load_data(stage='test', task='ranking')

In [3]:
type(train_data_pack)

matchzoo.data_pack.data_pack.DataPack

`DataPack` is a MatchZoo native data structure that most MatchZoo data handling processes build upon. A `DataPack` is consists of three `pandas.DataFrame`:

In [4]:
train_data_pack.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [5]:
train_data_pack.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [6]:
train_data_pack.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0
1,Q1,D1-1,0
2,Q1,D1-2,0
3,Q1,D1-3,1
4,Q1,D1-4,0


It is also possible to convert a `DataPack` into a single `pandas.DataFrame` that holds all information.

In [7]:
train_data_pack.frame().head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0


However, using such `pandas.DataFrame` consumes much more memory if there are many duplicates in the texts, and that is the exact reason why we use `DataPack`. For more details about data handling, consult `matchzoo/tutorials/data_handling.ipynb`.

# Preprocessing

MatchZoo preprocessors are used to convert a raw `DataPack` into a `DataPack` that ready to be fed into a model. 

In [8]:
preprocessor = mz.preprocessors.NaivePreprocessor()

There are two steps to use a preprocessor. First, `fit`. Then, `transform`. `fit` will only changes the preprocessor's inner state but not the input `DataPack`.

In [9]:
preprocessor.fit(train_data_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8350.28it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4362.91it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 418382/418382 [00:00<00:00, 2315810.08it/s]


`fit` will gather all information it needs into its `context`. In the above example, we can see a `VocabularyUnit` is built during the fitting process using `train_data_pack`.

In [10]:
preprocessor.context

{'vocab_unit': <matchzoo.processor_units.processor_units.VocabularyUnit at 0x7f301242cb38>}

`VocabularyUnit` is a `StatefulProcessorUnit` that has a similar `fit`/`transform` interface. Once a `VocabularyUnit` `fit`, it will store a mapping from `term` to `index` and the reverse in its `state`.

The `NaivePreprocessor` already handles `VocabularyUnit` internally, so we do not have to worry about that. Just access it through the `NaivePreprocessor`'s `context`.

In [11]:
vocab_unit = preprocessor.context['vocab_unit']
print(vocab_unit.state['term_index']['match'])
print(vocab_unit.state['term_index']['zoo'])
print(vocab_unit.state['index_term'][1])
print(vocab_unit.state['index_term'][2])

19073
10516
kickoff
mater


Once `fit`, the preprocessor has enough information to `transform`.  `transform` will not change the preprocessor's inner state and the input `DataPack`, but return a transformed `DataPack`.

In [12]:
train_data_pack_processed = preprocessor.transform(train_data_pack)
test_data_pack_processed = preprocessor.transform(test_data_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 2118/2118 [00:00<00:00, 7369.12it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4064.75it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 633/633 [00:00<00:00, 7218.68it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 5961/5961 [00:01<00:00, 3947.08it/s]


In [13]:
train_data_pack_processed.left.head()

,text_left
id_left,
Q1,"[3396, 10833, 4493, 23440, 23796, 0, 0, 0, 0, ..."
Q2,"[3396, 10833, 22650, 8313, 25495, 22650, 23052..."
Q5,"[3396, 15029, 1398, 2380, 23617, 0, 0, 0, 0, 0..."
Q6,"[3396, 5984, 1223, 22650, 14446, 11021, 21887,..."
Q7,"[3396, 6613, 9185, 14825, 18281, 23616, 21338,..."


As we can see, `text_left` is already in sequence form that nerual networks love.

Just to make sure we have the correct sequence:

In [14]:
print('Before:', train_data_pack.left.loc['Q1']['text_left'])
sequence = train_data_pack_processed.left.loc['Q1']['text_left']
print('After:', sequence)
print('Translated:', '_'.join([vocab_unit.state['index_term'][i] for i in sequence]))

Before: how are glacier caves formed?
After: [3396, 10833, 4493, 23440, 23796, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Translated: how_are_glacier_caves_formed_________________________


For more details about data handling, consult `matchzoo/tutorials/preprocessing.ipynb`.

# Build Model

MatchZoo provides many built-in text matching models.

In [15]:
mz.models.list_available()

[matchzoo.models.naive_model.NaiveModel,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline_model.DenseBaselineModel,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM]

In [16]:
model = mz.models.DenseBaselineModel()

The model is initialized with a hyper parameter table, in which values are partially filled.

In [17]:
print(model.params)

name                          None
model_class                   <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
input_shapes                  None
task                          None
optimizer                     None
with_multi_layer_perceptron   True
mlp_num_units                 256
mlp_num_layers                None
mlp_num_fan_out               None
mlp_activation_func           None


In [18]:
model.params['name'] = 'My First Model'
model.params['mlp_num_units'] = 3
print(model.params)

name                          My First Model
model_class                   <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
input_shapes                  None
task                          None
optimizer                     None
with_multi_layer_perceptron   True
mlp_num_units                 3
mlp_num_layers                None
mlp_num_fan_out               None
mlp_activation_func           None


Use `guess_and_fill_missing_params` to automatically fill-in other hyper parameters. This involves some guessing so the parameter it fills could be wrong. For example, the default task is `Ranking`, and if we do not set it to `Classification` manaully for data packs prepared for classification, then the shape of the model output and the data will mismatch.

In [19]:
model.guess_and_fill_missing_params()
print(model.params)

Parameter "task" set to Ranking Task.
Parameter "input_shapes" set to [(30,), (30,)].
Parameter "optimizer" set to adam.
Parameter "mlp_num_layers" set to 3.
Parameter "mlp_num_fan_out" set to 32.
Parameter "mlp_activation_func" set to relu.
name                          My First Model
model_class                   <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 3
mlp_num_layers                3
mlp_num_fan_out               32
mlp_activation_func           relu


In [20]:
model.params.completed()

True

With all parameters filled in, we can now build and compile the model.

In [21]:
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 30)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 60)           0           text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            183         concatenate_1[0][0]              
__________

For more details about models, consult `matchzoo/tutorials/models.ipynb`.

# Train, Evaluate, Predict

A `DataPack` can `unpack` itself into data that can be directly used to train a MatchZoo model.

In [22]:
x, y = train_data_pack_processed.unpack()
test_x, test_y = test_data_pack_processed.unpack()

In [23]:
model.fit(x, y, batch_size=32, epochs=5)

Epoch 1/5
20360/20360 [==============================] - 2s 95us/step - loss: 0.0546
Epoch 2/5
20360/20360 [==============================] - 2s 85us/step - loss: 0.0486
Epoch 3/5
20360/20360 [==============================] - 2s 80us/step - loss: 0.0486
Epoch 4/5
20360/20360 [==============================] - 2s 82us/step - loss: 0.0485
Epoch 5/5
20360/20360 [==============================] - 2s 84us/step - loss: 0.0485


An alternative to train a model is to use a `DataGenerator`. This might be useful for delaying expensive preprocessing steps or doing real-time data augmentation. For more details about `DataGenerator`, consult `matchzoo/tutorials/data_handling.ipynb`.

In [24]:
data_generator = mz.DataGenerator(train_data_pack_processed, batch_size=32)

In [25]:
model.fit_generator(data_generator, epochs=5, use_multiprocessing=True, workers=4)

Epoch 1/5
637/637 [==============================] - 2s 3ms/step - loss: 0.0485
Epoch 2/5
637/637 [==============================] - 2s 3ms/step - loss: 0.0486
Epoch 3/5
637/637 [==============================] - 2s 3ms/step - loss: 0.0485
Epoch 4/5
637/637 [==============================] - 2s 3ms/step - loss: 0.0486
Epoch 5/5
637/637 [==============================] - 2s 3ms/step - loss: 0.0485


In [26]:
model.evaluate(test_x, test_y)

{mean_average_precision(0): 0.24650797375811317}

In [27]:
model.predict(test_x)

array([[0.05138376],
       [0.05138376],
       [0.05138376],
       ...,
       [0.05138376],
       [0.05138376],
       [0.05138376]], dtype=float32)

# Automation

MatchZoo strives for ease of use, and package `matchzoo.auto` is a perfect example of that.

`matchzoo.auto.prepare` handles interaction among data, model, and preprocessor automatically. For example, some model like `DSSM` have dynamic input shapes based on the result of word hashing. Some models have an embedding layer which dimension is related to the data's vocabulary size. `prepare` takes care of all that and returns properly prepared model, data, and preprocessor for you.

In [28]:
model_ok, train_ok, preprocesor_ok = mz.auto.prepare(
    model=mz.models.DSSM(),
    data_pack=train_data_pack[:100]
)
test_ok = preprocesor_ok.transform(test_data_pack, verbose=0)
model_ok.fit(*train_ok.unpack(), batch_size=32)
model_ok.evaluate(*test_ok.unpack())

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 13/13 [00:00<00:00, 4171.52it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 100/100 [00:00<00:00, 3155.82it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 8523/8523 [00:00<00:00, 2179759.33it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 13/13 [00:00<00:00, 2874.17it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 100/100 [00:00<00:00, 2337.53it/s]


Parameter "name" set to DSSM.
Parameter "mlp_num_layers" set to 3.
Parameter "mlp_num_units" set to 64.
Parameter "mlp_num_fan_out" set to 32.
Parameter "mlp_activation_func" set to relu.
Epoch 1/1
100/100 [==============================] - 1s 7ms/step - loss: 0.0455


{mean_average_precision(0): 0.1497662563664794}

For more details about automation, consult `matchzoo/tutorials/automation.ipynb`.

# Full Example

In [29]:
model_classes = [
    mz.models.CDSSM,
    mz.models.DSSM,
    mz.models.DUET,
    mz.models.MVLSTM,
    mz.models.DRMMTKS
]

In [30]:
task = mz.tasks.Ranking(metrics=['map', 'ndcg'])
results = []
for model_class in model_classes:
    print(model_class)
    model = model_class()
    model.params['task'] = task
    model_ok, train_ok, preprocesor_ok = mz.auto.prepare(
        model=model,
        data_pack=train_data_pack[:2000],
        verbose=0
    )
    test_ok = preprocesor_ok.transform(test_data_pack, verbose=0)
    callback = mz.engine.callbacks.EvaluateAllMetrics(
        model_ok,
        *test_ok.unpack(),
        batch_size=1024,
        verbose=0
    )
    history = model_ok.fit(*train_ok.unpack(), batch_size=32, epochs=20, callbacks=[callback])
    results.append({'name': model_ok.params['name'], 'history': history})

<class 'matchzoo.models.cdssm.CDSSM'>
Epoch 1/20
2000/2000 [==============================] - 11s 6ms/step - loss: 0.0456
Epoch 2/20
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0333
Epoch 3/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0176
Epoch 4/20
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0077
Epoch 5/20
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0037
Epoch 6/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0036
Epoch 7/20
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0018
Epoch 8/20
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0019
Epoch 9/20
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0011
Epoch 10/20
2000/2000 [==============================] - 11s 6ms/step - loss: 0.0021
Epoch 11/20
2000/2000 [==============================] - 11s 6ms/step - loss: 0.0037
Epoch 12/20
2000/2000 [=============

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 196/196 [00:00<00:00, 7797.07it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit:  22%|██▏       | 444/1988 [00:00<00:00, 4437.29it/s]

<class 'matchzoo.models.duet.DUET'>


Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1988/1988 [00:00<00:00, 4333.36it/s]
Processing length_right with len: 100%|██████████| 5961/5961 [00:00<00:00, 610376.60it/s]


Epoch 1/20
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0918
Epoch 2/20
2000/2000 [==============================] - 2s 985us/step - loss: 0.0442
Epoch 3/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0401
Epoch 4/20
2000/2000 [==============================] - 2s 999us/step - loss: 0.0368
Epoch 5/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0274
Epoch 6/20
2000/2000 [==============================] - 2s 989us/step - loss: 0.0231
Epoch 7/20
2000/2000 [==============================] - 2s 990us/step - loss: 0.0185
Epoch 8/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0185
Epoch 9/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0175
Epoch 10/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0186
Epoch 11/20
2000/2000 [==============================] - 2s 978us/step - loss: 0.0176
Epoch 12/20
2000/2000 [==============================] - 2s 1ms/step - loss

...

Epoch 1/20
2000/2000 [==============================] - 7s 4ms/step - loss: 0.0450
Epoch 2/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0416
Epoch 3/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0223
Epoch 4/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0105
Epoch 5/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0062
Epoch 6/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0037
Epoch 7/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0031
Epoch 8/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0027
Epoch 9/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0026
Epoch 10/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0026
Epoch 11/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0026
Epoch 12/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0026
E

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 196/196 [00:00<00:00, 8301.69it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit:  22%|██▏       | 432/1988 [00:00<00:00, 4318.77it/s]

<class 'matchzoo.models.drmmtks.DRMMTKS'>


Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1988/1988 [00:00<00:00, 4516.05it/s]
Processing length_right with len: 100%|██████████| 5961/5961 [00:00<00:00, 606806.45it/s]


Epoch 1/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0480
Epoch 2/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0439
Epoch 3/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0324
Epoch 4/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0157
Epoch 5/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0068
Epoch 6/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0033
Epoch 7/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0020
Epoch 8/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0015
Epoch 9/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0015
Epoch 10/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0012
Epoch 11/20
2000/2000 [==============================] - 2s 1ms/step - loss: 9.9670e-04
Epoch 12/20
2000/2000 [==============================] - 2s 1ms/step - loss: 9.20

In [31]:
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.models.tools import HoverTool
import IPython

output_notebook()

charts = {
    metric: figure(
        title=str(metric),
        sizing_mode='scale_width',
        width=800, height=400
    ) for metric in results[0]['history'].history.keys()
}
hover_tool = HoverTool(tooltips=[
    ("x", "$x"),
    ("y", "$y")
])
for metric, sub_chart in charts.items():
    lines = {}
    for result, color in zip(results, bokeh.palettes.Category10[10]):
        x = result['history'].epoch
        y = result['history'].history[metric]
        lines[result['name']] = sub_chart.line(
            x, y, color=color, line_width=2, alpha=0.5, legend=result['name'])
        sub_chart.add_tools(hover_tool)
show(column(*charts.values()))
#export_png(column(*charts.values()), "quick_start_chart.png")

Loading BokehJS ...